# Multiclass SVM 구현

In [70]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

#IRIS 데이터 로드
iris =  sns.load_dataset('iris') 
X= iris.iloc[:,:4] #학습할데이터
y = iris.iloc[:,-1] #타겟
print(y)

# 1. Multiclass SVM을 직접 구현하시는 것입니다. 기본적으로 사이킷런에 있는 SVM은 멀티클래스 SVM을 지원하지만 과제에서는 절대 쓰면 안됩 니다! Iris 데이터는 총 세 개의 클래스가 있으므로 이 클래스를 one- hot인코딩한 뒤, 각각 binary SVM을 트레이닝하고 이 결과를 조합하여 multiclass SVM을 구현하시면 됩니다
# 2. 기본적으로 one vs one, one vs rest 방법이 있으며 둘 중 자유롭게 구현해주세요. 만약 투표결과가 동점으로 나온 경우(예를 들어, 각각의 SVM 결과가 A vs B 의 경우 A로 판별, B vs A 의 경우 B로 판별, C vs A 의 경우 C로 판별한 경우 투표를 통해 Class를 결정할 수 없음)
# 1) decision_function을 활용하시거나
# 2) 가장 개수가 많은 클래스를 사용하시거나
# 3) 랜덤으로 하나를 뽑거나 하는 방법 등을 이용해 동점자인 경우를 판별 해주시면 됩니다.


0         setosa
1         setosa
2         setosa
3         setosa
4         setosa
         ...    
145    virginica
146    virginica
147    virginica
148    virginica
149    virginica
Name: species, Length: 150, dtype: object


In [71]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)

In [72]:
def standardization(train, test):
    scaler = StandardScaler()
    train = scaler.fit_transform(train)
    test = scaler.transform(test)
    return train, test

X_train, X_test = standardization(X_train, X_test)

In [73]:
X_train

array([[ 0.78522493,  0.32015325,  0.77221097,  1.04726529],
       [-0.26563371, -1.29989934,  0.0982814 , -0.11996537],
       [ 0.43493872,  0.78302542,  0.94069336,  1.43634218],
       [-0.84944407,  0.78302542, -1.24957775, -1.28719604],
       [-0.38239578, -1.7627715 ,  0.15444219,  0.13941922],
       [ 0.55170079, -0.374155  ,  1.05301496,  0.7878807 ],
       [ 0.31817664, -0.14271892,  0.65988937,  0.7878807 ],
       [ 0.20141457, -0.374155  ,  0.43524618,  0.39880381],
       [-1.66677857, -0.14271892, -1.36189934, -1.28719604],
       [-0.14887164, -0.60559109,  0.21060299,  0.13941922],
       [-0.14887164, -1.06846325, -0.12636179, -0.24965767],
       [ 0.31817664, -0.60559109,  0.15444219,  0.13941922],
       [ 0.66846286, -0.83702717,  0.88453256,  0.91757299],
       [ 0.0846525 , -0.14271892,  0.77221097,  0.7878807 ],
       [-0.49915786, -0.14271892,  0.43524618,  0.39880381],
       [-0.26563371, -0.60559109,  0.65988937,  1.04726529],
       [ 2.18636979,  1.

In [74]:
X_test

array([[-0.14887164, -0.374155  ,  0.26676379,  0.13941922],
       [ 0.31817664, -0.60559109,  0.54756778,  0.00972692],
       [ 0.31817664, -1.06846325,  1.05301496,  0.26911151],
       [-1.5500165 , -1.7627715 , -1.36189934, -1.15750374],
       [ 0.0846525 ,  0.32015325,  0.60372857,  0.7878807 ],
       [ 0.78522493, -0.14271892,  0.99685416,  0.7878807 ],
       [-0.84944407,  1.70876975, -1.24957775, -1.15750374],
       [ 0.20141457, -0.14271892,  0.60372857,  0.7878807 ],
       [-0.38239578,  2.63451409, -1.30573855, -1.28719604],
       [-0.38239578, -1.29989934,  0.15444219,  0.13941922],
       [ 0.66846286,  0.08871717,  0.99685416,  0.7878807 ],
       [-0.38239578,  1.0144615 , -1.36189934, -1.28719604],
       [-0.49915786,  0.78302542, -1.13725615, -1.28719604],
       [ 0.43493872, -0.60559109,  0.60372857,  0.7878807 ],
       [ 0.55170079, -1.7627715 ,  0.37908538,  0.13941922],
       [ 0.55170079,  0.55158933,  0.54756778,  0.52849611],
       [-1.19973028,  0.

In [75]:
from sklearn.metrics.pairwise import rbf_kernel 

### OVR

In [86]:
def fit(X_train, y_train, num_classes, kernel, C, gamma):
    y_train = pd.get_dummies(y_train) #One Hot Encoding
    clfs = [SVC(kernel = kernel, C = C, gamma = gamma) for _ in range(num_classes)]
    for i in range(num_classes):
        clfs[i].fit(X_train,y_train.iloc[:,i]) 
    classes = y_train.columns
            
    return classes, clfs

In [87]:
def predict(X_test, classes, clfs):
        pred_df = pd.DataFrame([svm.predict(X_test) for svm in clfs]).T # 각 클래스 별 예측값
        decisions = np.array([svm.decision_function(X_test) for svm in clfs]).T # 각 클래스 별 거리
        
        final_pred = []
        for i in range(len(pred_df)):

            if sum(pred_df.iloc[i]) == 1:
                label = pred_df.iloc[i][pred_df.iloc[i] == 1].index[0]
                final_pred.append(classes[label])
            
            else:
                label = np.argmax(decisions[i]) # 동점 나온 경우 decision_function을 활용
                final_pred.append(classes[label])
        
        return final_pred

In [97]:
classes, clfs = fit(X_train = X_train, y_train = y_train, num_classes = 3, kernel = 'rbf', C= 3, gamma = 5)
ovr_pred= predict(X_test, classes, clfs)


In [98]:
accuracy_score(y_test, ovr_pred)

0.8666666666666667

### OVO

In [99]:
from itertools import combinations
from sklearn.datasets import load_digits

In [100]:
def ovo_fit(X_train, y_train,num_classes, kernel, C, gamma):
    classes = y_train.unique()
    i = 0
    combi = []
    clfs = [{'class' : None,'clf' : SVC(kernel = kernel, C = C, gamma = gamma)} for _ in range(int(num_classes * (num_classes-1) / 2))]
    for c in combinations(classes, 2):
        idx = (y_train == c[0]) | (y_train == c[1])
        clfs[i]['clf'].fit(X_train[idx], y_train[idx])
        clfs[i]['class'] = c
        combi.append(c)
        i += 1
    return combi, clfs

In [101]:
def ovo_predict(X_test, combi, clfs):
    #
    preds_df = pd.DataFrame([svm['clf'].predict(X_test) for svm in clfs]).T 
    decisions = pd.DataFrame([svm['clf'].decision_function(X_test) for svm in clfs]).T
    decisions.columns = combi
        
    final_pred = []
    for i in range(len(preds_df)):
            
        if preds_df.iloc[i].value_counts().iloc[0] > preds_df.iloc[i].value_counts().iloc[1]:
            label = (preds_df.iloc[i].value_counts() / len(preds_df.iloc[i])).index[0]
            final_pred.append(label)

        else: # 동점 나온 경우 decision_function을 활용
            decision_for_row = {key : 0 for key in classes}
            for c, d in zip(decisions.iloc[i].index, decisions.iloc[i]):
                if d > 0:
                    decision_for_row[c[0]] += d
                else:
                    decision_for_row[c[1]] -= d
            label = sorted(decision_for_row.items(), key = lambda x : x[1], reverse = True)[0][0]
            final_pred.append(label)
    return final_pred

In [102]:
combi, clfs = ovo_fit(X_train = X_train, y_train = y_train, num_classes = 3, kernel = 'rbf', C= 3, gamma = 5)
ovo_pred= ovo_predict(X_test, combi, clfs)

In [103]:
accuracy_score(y_test, ovo_pred)

0.8666666666666667